# import

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
import math
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
from  PIL import Image
import os
import random
from tqdm import tqdm
import timm
import sys
sys.path.insert(0, '../input/tiny-vit-model')
import tiny_vit

# 定义参数

In [2]:
TRAIN_CSV_PATH = '../input/cassava-leaf-disease-classification/train.csv'
TEST_IMAGE_PATH = '../input/cassava-leaf-disease-classification/test_images/'
SUBMISSION_PATH = 'submission.csv'
RES50_PATH = '../input/ensemble-1023/1022_res50.pth'
B4NS_PATH = '../input/ensemble-1023/1022_b4ns.pth'
TINY_VIT_PATH = '../input/1103-tinyvit/tinyvit.pth'
CROPNET_PATH  = '../input/cropnet/cropnet'
DEVICES = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
OUT_CLASSES = 5
IMAGE_SIZE = 512
SEED = 42
TTA_1 = 5
TTA_2 = 3
TTA_3 = 3

# tr5.seed

In [7]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
seed_everything(SEED)

# 定义模型

In [8]:
model_name1 = "resnext50_32x4d"
my_model_1 = timm.create_model(model_name1, pretrained=False)
my_model_1.fc = nn.Linear(my_model_1.fc.in_features, OUT_CLASSES)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act2): ReLU(inplace=True)
      (aa): Identity()
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_

In [9]:
model_name2 = "tf_efficientnet_b4_ns"
my_model_2 = timm.create_model(model_name2, pretrained=False)
my_model_2.classifier = nn.Linear(my_model_2.classifier.in_features, OUT_CLASSES)

/opt/conda/lib/python3.10/site-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name tf_efficientnet_b4_ns to current tf_efficientnet_b4.ns_jft_in1k.
  model = create_fn(


EfficientNet(
  (conv_stem): Conv2dSame(3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNormAct2d(
    48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
        (bn1): BatchNormAct2d(
          48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
          24, ep

In [4]:
my_model_3 = tiny_vit.tiny_vit_21m_512(pretrained=False)
my_model_3.head = nn.Linear(my_model_3.head.in_features, OUT_CLASSES)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act2): ReLU(inplace=True)
      (aa): Identity()
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_

In [10]:
torch.cuda.empty_cache()

# 验证

In [11]:
test_augs = A.Compose([
    A.OneOf([
        A.Resize(IMAGE_SIZE, IMAGE_SIZE, p=1.0),
        A.CenterCrop(IMAGE_SIZE, IMAGE_SIZE, p=1.0),
        A.RandomResizedCrop(IMAGE_SIZE, IMAGE_SIZE, p=1.0)
    ], p=1.0),
    A.Transpose(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Resize(IMAGE_SIZE, IMAGE_SIZE),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
    ToTensorV2(p=1.0)
    ], p=1.0
)

IMAGE_SIZE1 = 512
IMAGE_SIZE2 = 224
test_augs_tf = A.Compose([A.RandomResizedCrop(IMAGE_SIZE1, IMAGE_SIZE1),
                   A.Resize(IMAGE_SIZE2, IMAGE_SIZE2)])

preds_1 = []
model_param = torch.load(RES50_PATH)
new_model_param = {k[7:]: v for k, v in model_param.items() if 'module.' in k}
my_model_1.load_state_dict(new_model_param)
my_model_1 = my_model_1.to(DEVICES[0])
my_model_1.eval()
test_image_list = np.asarray([image_name for image_name in os.listdir(TEST_IMAGE_PATH)])
for single_image_name in test_image_list:
    with torch.no_grad():
        ans = torch.zeros(5).cuda()
        for _ in range(1):
            image = Image.open(os.path.join(TEST_IMAGE_PATH, single_image_name))
            aug_image = test_augs(image=np.array(image))['image']
            test_image = torch.tensor(aug_image, dtype=torch.float).unsqueeze(0).cuda()
            ans += my_model_1(test_image).view(ans.shape)
        preds_1.append(ans)
predictions_1 = torch.stack(preds_1, dim=0).to('cpu')
normalize_pred_1 = F.normalize(predictions_1.T, p=2, dim=0).T
torch.cuda.empty_cache()


preds_2 = []
model_param = torch.load(B4NS_PATH)
new_model_param = {k[7:]: v for k, v in model_param.items() if 'module.' in k}
my_model_2.load_state_dict(new_model_param)
my_model_2 = my_model_2.to(DEVICES[0])
my_model_2.eval()
test_image_list = np.asarray([image_name for image_name in os.listdir(TEST_IMAGE_PATH)])
for single_image_name in test_image_list:
    with torch.no_grad():
        ans = torch.zeros(5).cuda()
        for _ in range(TTA_1):
            image = Image.open(os.path.join(TEST_IMAGE_PATH, single_image_name))
            aug_image = test_augs(image=np.array(image))['image']
            test_image = torch.tensor(aug_image, dtype=torch.float).unsqueeze(0).cuda()
            ans += my_model_2(test_image).view(ans.shape)
        ans /= TTA_1
        preds_2.append(ans)
predictions_2 = torch.stack(preds_2, dim=0).to('cpu')
normalize_pred_2 = F.normalize(predictions_2.T, p=2, dim=0).T
normalize_pred_2 = normalize_pred_2.cpu()
torch.cuda.empty_cache()


preds_3 = []
model_param = torch.load(TINY_VIT_PATH)
new_model_param = {k[7:]: v for k, v in model_param.items() if 'module.' in k}
my_model_3.load_state_dict(new_model_param)
my_model_3 = my_model_3.to(DEVICES[0])
my_model_3.eval()
test_image_list = np.asarray([image_name for image_name in os.listdir(TEST_IMAGE_PATH)])
for single_image_name in test_image_list:
    with torch.no_grad():
        ans = torch.zeros(5).cuda()
        for _ in range(TTA_2):
            image = Image.open(os.path.join(TEST_IMAGE_PATH, single_image_name))
            aug_image = test_augs(image=np.array(image))['image']
            test_image = torch.tensor(aug_image, dtype=torch.float).unsqueeze(0).cuda()
            ans += my_model_3(test_image).view(ans.shape)
        ans /= TTA_2
        preds_3.append(ans)
predictions_3 = torch.stack(preds_3, dim=0).to('cpu')
normalize_pred_3 = F.normalize(predictions_3.T, p=2, dim=0).T
torch.cuda.empty_cache()

import tensorflow as tf
my_model_4 = tf.keras.models.load_model(CROPNET_PATH)
preds_4 = []
test_image_list = np.asarray([image_name for image_name in os.listdir(TEST_IMAGE_PATH)])
for single_image_name in test_image_list:
    ans = torch.zeros(5)
    for _ in range(TTA_3):
        image = Image.open(os.path.join(TEST_IMAGE_PATH, single_image_name))
        test_image = np.array([test_augs_tf(image=np.asarray(image))['image']]) / 255.0
        ans += (my_model_4(test_image).numpy())[0][0: 5]
    ans /= TTA_3
    preds_4.append(ans)
predictions_4 = torch.stack(preds_4, dim=0)
normalize_pred_4 = F.normalize(predictions_4.T, p=2, dim=0).T
torch.cuda.empty_cache()

final_pred = (normalize_pred_1 * 0.2) + (normalize_pred_2 * 0.2) + (normalize_pred_3 * 0.4) + (normalize_pred_4 * 0.2)

label = final_pred.argmax(dim=-1).numpy()
label_list = list(label)
df_submission = pd.DataFrame(columns=pd.read_csv(TRAIN_CSV_PATH).columns)
df_submission['image_id'] = pd.DataFrame(test_image_list)
df_submission['label'] = pd.DataFrame(label_list)
df_submission.to_csv(SUBMISSION_PATH, index=False)

/tmp/ipykernel_26/3606756552.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_image = torch.tensor(aug_image, dtype=torch.float).unsqueeze(0).cuda()
/tmp/ipykernel_26/3606756552.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_image = torch.tensor(aug_image, dtype=torch.float).unsqueeze(0).cuda()
